<a href="https://colab.research.google.com/github/LlakmalGamage/Translator-Using-Transformers/blob/main/translator_spanish_version_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Preprocessing

In [1]:
import pandas as pd

In [ ]:
# german_file_path="Data_Sets/deu.txt"
# spanish_file_path="Data_Sets/data.csv"

In [ ]:
# Initialize lists to store German and English data
# german_data = []
# english_data = []

# with open(german_file_path,'r',encoding='utf-8') as file:
#     for line in file:
#         english,german,_=line.strip().split('\t')
#         english_data.append(english)
#         german_data.append(german)


# # Create a DataFrame using Pandas
# df = pd.DataFrame({ 'English': english_data , 'German': german_data})

# # Display the first few rows of the DataFrame
# print(df[:20])

In [ ]:
# df_spanish=pd.read_csv(spanish_file_path)

# # renaming the names
# new_column_names={'english':'English','spanish':'Spanish'}
# df_spanish=df_spanish.rename(columns=new_column_names)

# print(df_spanish[:10])

In [ ]:
# file_name=f"data_german.csv"

# df.to_csv(f"Data_Sets/{file_name}",index=False)
# print("DataFrame saved successfully as", file_name)

In [2]:
import os
# current_file_path="Data_Sets/data.csv"
# new_file_path="Data_Sets/data_spanish.csv"

# os.rename(current_file_path,new_file_path)

# print("File renamed successfully to", new_file_path)

In [3]:
import pandas as pd
# german_file_path="data_german.csv"
spanish_file_path="data_spanish.csv"

In [4]:
# df1=pd.read_csv(german_file_path)
df2=pd.read_csv(spanish_file_path)

In [5]:
import random
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
import string
import re
import csv



In [7]:
df2.head(20)

,english,spanish
0,Go.,Ve.
1,Go.,Vete.
2,Go.,Vaya.
3,Go.,Váyase.
4,Hi.,Hola.
5,Run!,¡Corre!
6,Run.,Corred.
7,Who?,¿Quién?
8,Fire!,¡Fuego!
9,Fire!,¡Incendio!


In [9]:
first_column_values_last_10_rows=df2['english'].tail(10)


paragraph = '\n'.join(first_column_values_last_10_rows)
# Print the paragraph
print(paragraph)

You can't view Flash content on an iPad. However, you can easily email yourself the URLs of these web pages and view that content on your regular computer when you get home.
A mistake young people often make is to start learning too many languages at the same time, as they underestimate the difficulties and overestimate their own ability to learn them.
No matter how much you try to convince people that chocolate is vanilla, it'll still be chocolate, even though you may manage to convince yourself and a few others that it's vanilla.
In 1969, Roger Miller recorded a song called "You Don't Want My Love." Today, this song is better known as "In the Summer Time." It's the first song he wrote and sang that became popular.
A child who is a native speaker usually knows many things about his or her language that a non-native speaker who has been studying for years still does not know and perhaps will never know.
There are four main causes of alcohol-related death. Injury from car accidents or v

In [10]:
len(df2)

118964

# Split the English and german translation pairs

In [11]:
#common function for both spanish and german

class split_pairs:
 def split_pairs_method(self,df1):
  text_pairs=[]

  for i in range(len(df1)):
    english,language=df1["english"][i],df1["spanish"][i]
    language="[start] " + language + " [end]"
    text_pairs.append((english,language))

  return text_pairs


In [12]:
#randomly selecting that if the above function work
class random_pair_test:
  def random_test_method(self,text_pairs):
   for i in range(3):
    print(random.choice(text_pairs))


In [13]:
#pairing for german text

spanish_text_pairs=split_pairs().split_pairs_method(df2)

random_pair_test().random_test_method(spanish_text_pairs)

('The bag was too heavy for me to carry by myself.', '[start] La bolsa era muy pesada para llevarla yo solo. [end]')
('Tom felt uneasy talking to Mary about that matter.', '[start] Tom se sentía incomodo de hablar con Mary acerca de ese asunto. [end]')
('I get you.', '[start] Ya te pillo. [end]')


# Randomizing the data

In [14]:
random.shuffle(spanish_text_pairs)

# Split the data into training, validation,testing

In [15]:
#class for splitting text pairs in to train,test,validation
class splitting:
    def splitting_method(self,text_pairs):
        num_val_sample=int(0.15*len(text_pairs))
        num_train_samples=len(text_pairs) - 2 * num_val_sample
        train_pairs=text_pairs[:num_train_samples]
        val_pairs=text_pairs[num_train_samples:num_train_samples+num_val_sample]
        test_pairs=text_pairs[num_train_samples+num_val_sample:]

        print("Total Sentences: ",len(text_pairs))
        print("Training set size: ",len(train_pairs))
        print("Validation set size: ",len(val_pairs))
        print("Testng set size: ",len(test_pairs))
        return train_pairs,val_pairs,test_pairs



In [16]:
spanish_train_pairs,spanish_val_pairs,spanish_test_pairs=splitting().splitting_method(spanish_text_pairs)


Total Sentences:  118964
Training set size:  83276
Validation set size:  17844
Testng set size:  17844


In [17]:
print(len(spanish_test_pairs)+len(spanish_train_pairs)+len(spanish_val_pairs))

118964


In [18]:
print(spanish_val_pairs[200])

('I really like hard boiled eggs.', '[start] Me gustan mucho los huevos duros. [end]')


# Removing Puncuations

In [19]:
strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")


f"[{re.escape(strip_chars)}]"

f"{5+3}"

'8'

# Vectorizing the English and spanish text pairs

In [20]:
def custom_standardization(input_string):
    lowercase=tf.strings.lower(input_string)
    return tf.strings.regex_replace(lowercase,f"[{re.escape(strip_chars)}]", "")

vocab_size=15000
sequence_length=20

source_vectorization=layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
)

target_vectorization=layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length+1,
    standardize=custom_standardization,
)

train_english_texts=[pair[0] for pair in spanish_train_pairs]
train_spanish_texts=[pair[1] for pair in spanish_train_pairs]

source_vectorization.adapt(train_english_texts)
target_vectorization.adapt(train_spanish_texts)

In [21]:
print(train_english_texts[1])
print(train_spanish_texts[1])

print(source_vectorization)

Monroe received 65 votes.
[start] Monroe recibió 65 votos. [end]


In [22]:
print(spanish_train_pairs[1:5])

[('Monroe received 65 votes.', '[start] Monroe recibió 65 votos. [end]'), ("I've lost my strength.", '[start] Perdí mi fuerza. [end]'), ('As soon as I woke up, the headache returned.', '[start] En cuanto me desperté, volvió el dolor de cabeza. [end]'), ('I was invited by an old friend.', '[start] Me invitó un viejo amigo. [end]')]


# Preparing datasets for the translation task

In [24]:
batch_size=64

def format_dataset(eng,spn):
    eng=source_vectorization(eng)
    spn=target_vectorization(spn)
    return ({"english":eng,
             "spanish":spn[:,:-1],
             },spn[:,1:])

def make_dataset(pairs):
    eng_texts,spn_texts=zip(*pairs)
    eng_texts=list(eng_texts)
    spn_texts=list(spn_texts)
    dataset=tf.data.Dataset.from_tensor_slices((eng_texts,spn_texts))
    dataset=dataset.batch(batch_size)
    dataset=dataset.map(format_dataset,num_parallel_calls=4)

    return dataset.shuffle(2048).prefetch(16).cache()

train_data=make_dataset(spanish_train_pairs)
val_data=make_dataset(spanish_val_pairs)

# print(train_data)
# print(val_data)

for inputs, targets in train_data.take(1):
    print(f"inputs['english'].shape: {inputs['english'].shape}")
    print(f"inputs['spanish'].shape: {inputs['spanish'].shape}")
    print(f"targets.shape: {targets.shape}")



inputs['english'].shape: (64, 20)
inputs['spanish'].shape: (64, 20)
targets.shape: (64, 20)


In [25]:
print(list(train_data.as_numpy_iterator())[50])

({'english': array([[   3,  388, 1280, ...,    0,    0,    0],
       [  12, 9910, 3237, ...,    0,    0,    0],
       [  21, 1001,    2, ...,    0,    0,    0],
       ...,
       [  87,   18,   63, ...,    0,    0,    0],
       [   6,   65,  534, ...,    0,    0,    0],
       [  51,  130,   17, ...,    0,    0,    0]]), 'spanish': array([[    2,   510,     6, ...,     0,     0,     0],
       [    2,   106, 10356, ...,     0,     0,     0],
       [    2,    25,   921, ...,     0,     0,     0],
       ...,
       [    2,   721,   167, ...,     0,     0,     0],
       [    2,     8,     7, ...,     0,     0,     0],
       [    2,    27,   453, ...,     0,     0,     0]])}, array([[  510,     6,    31, ...,     0,     0,     0],
       [  106, 10356,    22, ...,     0,     0,     0],
       [   25,   921,     9, ...,     0,     0,     0],
       ...,
       [  721,   167,  1026, ...,     0,     0,     0],
       [    8,     7,   135, ...,     0,     0,     0],
       [   27,   45

# Transformers encoder implemented as a subclassed Layer

In [26]:
class TransformerEncoder(layers.Layer):
    def __init__(self,embed_dim,dense_dim,num_heads,**kwargs):
        super().__init__(**kwargs)
        self.embed_dim=embed_dim
        self.dense_dim=dense_dim
        self.num_heads=num_heads
        self.attention=layers.MultiHeadAttention(
           num_heads=num_heads,key_dim=embed_dim)
        self.dense_proj=keras.Sequential(
            [layers.Dense(dense_dim,activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1=layers.LayerNormalization()
        self.layernorm_2=layers.LayerNormalization()

    def call(self,inputs,mask=None):
        if mask is not None:
            mask=mask[:,tf.newaxis,:]
        attention_output=self.attention(
            inputs,inputs,attention_mask=mask
        )
        project_input=self.layernorm_1(inputs+attention_output)
        project_output=self.dense_proj(project_input)

        return self.layernorm_2(project_input+project_output)

    def get_config(self):
        config=super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

# Transformer decorder

In [27]:
class TransformerDecoder(layers.Layer):
    def __init__(self,embed_dim,dense_dim,num_heads,**kwargs):
        super().__init__(**kwargs)
        self.embed_dim=embed_dim
        self.dense_dim=dense_dim
        self.num_heads=num_heads
        self.attention_1=layers.MultiHeadAttention(
            num_heads=num_heads,key_dim=embed_dim)
        self.attention_2=layers.MultiHeadAttention(
            num_heads=num_heads,key_dim=embed_dim)
        self.dense_proj=keras.Sequential(
            [layers.Dense(dense_dim,activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1=layers.LayerNormalization()
        self.layernorm_2=layers.LayerNormalization()
        self.layernorm_3=layers.LayerNormalization()
        self.supports_masking=True

    def get_config(self):
        config=super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

    def get_casual_attention_mask(self,inputs):
        input_shape=tf.shape(inputs)
        batch_size,sequence_length=input_shape[0],input_shape[1]
        i=tf.range(sequence_length)[:,tf.newaxis]
        j=tf.range(sequence_length)
        mask=tf.cast(i>=j,dtype="int32")
        mask=tf.reshape(mask,(1,input_shape[1],input_shape[1]))
        mult=tf.concat(
            [tf.expand_dims(batch_size,-1),
             tf.constant([1,1],dtype=tf.int32)],axis=0)

        return tf.tile(mask,mult)


    def call(self,inputs,encorder_outputs,mask=None):
        casual_mask=self.get_casual_attention_mask(inputs)
        if mask is not None:
            padding_mask=tf.cast(
                mask[:,tf.newaxis,:],dtype="int32"
            )
            padding_mask=tf.minimum(padding_mask,casual_mask)
        else:
            padding_mask=mask
        attention_output_1=self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            attention_mask=casual_mask
        )
        attention_output_1=self.layernorm_1(inputs+attention_output_1)
        attention_output_2=self.attention_2(
            query=attention_output_1,
            value=encorder_outputs,
            key=encorder_outputs,
            attention_mask=padding_mask,
        )
        attention_output_2=self.layernorm_2(
            attention_output_1+attention_output_2
        )
        proj_output=self.dense_proj(attention_output_2)
        return self.layernorm_3(attention_output_2+proj_output)




# Positional Encoding

In [28]:
class PositionalEmbedding(layers.Layer):
    def __init__(self,sequence_length,input_dim,output_dim,**kwargs):
        super().__init__(**kwargs)
        self.token_embeddings=layers.Embedding(
            input_dim=input_dim,output_dim=output_dim)
        self.position_embeddings=layers.Embedding(
            input_dim=sequence_length,output_dim=output_dim)
        self.sequence_length=sequence_length
        self.input_dim=input_dim
        self.output_dim=output_dim

    def call(self,inputs):
        length=tf.shape(inputs)[-1]
        positions=tf.range(start=0,limit=length,delta=1)
        embedded_tokens=self.token_embeddings(inputs)
        embedded_positions=self.position_embeddings(positions)

        return embedded_tokens + embedded_positions


    def compute_mask(self, inputs, mask=None):
        return tf.not_equal(inputs, 0)

    def get_config(self):
        config=super(PositionalEmbedding,self).get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config

In [ ]:
# from tensorflow.python.framework.ops import disable_eager_execution
# disable_eager_execution()

# End-to-End Transformer

In [34]:
embed_dim=256
dense_dim=2048
num_heads=8

encoder_inputs=keras.Input(shape=(None,), dtype="int64", name="english")
x=PositionalEmbedding(sequence_length,vocab_size,embed_dim)(encoder_inputs)
encoder_outputs=TransformerEncoder(embed_dim,dense_dim,num_heads)(x)

decorder_inputs=keras.Input(shape=(None,),dtype="int64",name="spanish")
x=PositionalEmbedding(sequence_length,vocab_size,embed_dim)(decorder_inputs)
x=TransformerDecoder(embed_dim,dense_dim,num_heads)(x,encoder_outputs)
x=layers.Dropout(0.5)(x)
decorder_outputs=layers.Dense(vocab_size,activation="softmax")(x)
transformer_spanish=keras.Model([encoder_inputs,decorder_inputs],decorder_outputs)

In [35]:
transformer_spanish.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 english (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 spanish (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 positional_embedding_4 (Po  (None, None, 256)            3845120   ['english[0][0]']             
 sitionalEmbedding)                                                                               
                                                                                                  
 positional_embedding_5 (Po  (None, None, 256)            3845120   ['spanish[0][0]']       

# Training the sequence-to-sequence Transformer

In [36]:
transformer_spanish.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

transformer_spanish.fit(train_data,epochs=50,validation_data=val_data)

Epoch 1/50
1302/1302 [==============================] - 112s 79ms/step - loss: 3.7930 - accuracy: 0.4407 - val_loss: 2.8610 - val_accuracy: 0.5415
Epoch 2/50
1302/1302 [==============================] - 90s 69ms/step - loss: 2.8522 - accuracy: 0.5496 - val_loss: 2.5043 - val_accuracy: 0.5910
Epoch 3/50
1302/1302 [==============================] - 89s 68ms/step - loss: 2.5544 - accuracy: 0.5933 - val_loss: 2.3742 - val_accuracy: 0.6160
Epoch 4/50
1302/1302 [==============================] - 89s 68ms/step - loss: 2.3933 - accuracy: 0.6196 - val_loss: 2.3345 - val_accuracy: 0.6260
Epoch 5/50
1302/1302 [==============================] - 90s 69ms/step - loss: 2.2879 - accuracy: 0.6384 - val_loss: 2.2980 - val_accuracy: 0.6333
Epoch 6/50
1302/1302 [==============================] - 89s 69ms/step - loss: 2.2159 - accuracy: 0.6514 - val_loss: 2.2875 - val_accuracy: 0.6376
Epoch 7/50
1302/1302 [==============================] - 89s 69ms/step - loss: 2.1551 - accuracy: 0.6632 - val_loss: 2.2880 

In [37]:
import numpy as np

ger_vocab = target_vectorization.get_vocabulary()
ger_index_lookup = dict(zip(range(len(ger_vocab)), ger_vocab))
max_decoded_sentence_length = 20

def decode_sequence(input_sentence):
    tokenized_input_sentence = source_sentence = source_vectorization([input_sentence])
    decoded_sentence = "[start]"

    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = target_vectorization([decoded_sentence])[:, :-1]
        predictions = transformer_spanish([tokenized_input_sentence, tokenized_target_sentence])
        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = ger_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token
        if sampled_token == "[end]":
            break

    return decoded_sentence

In [38]:
ger_eng_texts = [pair[0] for pair in spanish_test_pairs]
for _ in range(20):
    input_sentence = random.choice(ger_eng_texts)
    print("-")
    print(input_sentence)
    print(decode_sequence(input_sentence))

-
Where do you think the safest place would be?
[start] dónde crees que los días el lugar sería menos [end]  [end] [end] [end] [end] [end] [end]   [end]
-
I am a big fan of the arts.
[start] soy un [UNK] grande en la [UNK] [end]  [end] [end] [end] [end] [end] [end]   [end]  [end]
-
I fell asleep before my father came home.
[start] me quedé dormido antes de que mi padre venga en casa [end]  [end] [end] [end] [end]   
-
Just because he likes painting doesn't mean he's good at it.
[start] en realidad él le gusta compartir no quiere ser bueno así [end]  [end] [end] [end] [end] [end]  [end]
-
How is that possible?
[start] cómo es eso posible [end]  [end] [end] [end] [end] [end] [end]   [end]  [end]   
-
I'm so confused.
[start] estoy tan confundida [end]  [end] [end] [end] [end] [end] [end]    [end]     [end]
-
I've been preparing.
[start] me he estado haciendo [end]  [end] [end] [end] [end] [end] [end]   [end]     [end]
-
Come home with me.
[start] venga a casa conmigo [end]  [end] [end] [

In [39]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [40]:
transformer_spanish.save('/content/drive/My Drive/Translators/spanish_model')